# Deep Learning - Project

## Parking Space Detection

### Group 02

#### Rúben Torres fc62531
#### João Martins fc62532

In [ ]:
# RUN ONLY IN Google Colab
#!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!unzip /content/drive/MyDrive/datasets -d .

# 0. Imports & Global Constants

In [ ]:
import os
import random
import shutil
import xml.etree.ElementTree as ET

ORIGINAL_DATASET_PATH = "./datasets/original"
TRANSFORMED_DATASET_PATH = "./datasets/transformed"
TRAINING_DATASET_PATH = "./datasets/train"
VALIDATION_DATASET_PATH = "./datasets/val"
TESTING_DATASET_PATH = "./datasets/test"

TRAINING_RATIO = 0.7
VALIDATION_RATIO = 0.15
TESTING_RATIO = 0.15

GITKEEP = ".gitkeep"
DS_STORE = ".DS_Store"

IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 720

CLASS_MAPPING = {"0": 0, "1": 1}

# 1. Data Pipeline

# 1.1. Transforming

First we start by transforming our original dataset

In [ ]:
for root, _, files in os.walk(ORIGINAL_DATASET_PATH):
    for file in files:
        if file == GITKEEP or file == DS_STORE:
            continue
        source = os.path.join(root, file)
        destination = os.path.join(TRANSFORMED_DATASET_PATH, file)
        # print(destination)
        shutil.move(source, destination)

for dir in os.listdir(ORIGINAL_DATASET_PATH):
    if not os.path.isdir(dir):
        continue
    path = os.path.join(ORIGINAL_DATASET_PATH, dir)
    shutil.rmtree(path)

print("Merge complete!")

## 1.2. Create the Labels

We'll convert the labels from XML to TXT

In [ ]:
xml_files = [
    file for file in os.listdir(TRANSFORMED_DATASET_PATH) if file.endswith(".xml")
]

for xml_file in xml_files:
    xml_path = os.path.join(TRANSFORMED_DATASET_PATH, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    txt_filename = os.path.splitext(xml_file)[0] + ".txt"
    txt_path = os.path.join(TRANSFORMED_DATASET_PATH, txt_filename)
    with open(txt_path, "w") as txt_file:
        for space in root.findall("space"):
            occupied = space.get("occupied")
            class_index = CLASS_MAPPING.get(occupied, -1)
            if class_index == -1:
                continue
            rotated_rect = space.find("rotatedRect")
            center = rotated_rect.find("center")
            size = rotated_rect.find("size")
            x = float(center.get("x")) / IMAGE_WIDTH
            y = float(center.get("y")) / IMAGE_HEIGHT
            w = float(size.get("w")) / IMAGE_WIDTH
            h = float(size.get("h")) / IMAGE_HEIGHT
            txt_file.write(f"{class_index} {x} {y} {w} {h}\n")
    os.remove(xml_path)

print("Conversion complete!")

### Remove invalid files

In [ ]:
def is_valid_file(file):
    filename = os.path.splitext(file)[0]
    jpg_filename = filename + ".jpg"
    txt_filename = filename + ".txt"
    jpg_path = os.path.join(TRANSFORMED_DATASET_PATH, jpg_filename)
    txt_path = os.path.join(TRANSFORMED_DATASET_PATH, txt_filename)
    return os.path.isfile(jpg_path) and os.path.isfile(txt_path)


for file in os.listdir(TRANSFORMED_DATASET_PATH):
    if file == GITKEEP or file == DS_STORE:
        continue
    if not is_valid_file(file):
        file_path = os.path.join(TRANSFORMED_DATASET_PATH, file)
        os.remove(file_path)
        print(f"Removed file: {file_path}")

print("Removal complete!")

# Train Test Split

In [ ]:
folders = [TRAINING_DATASET_PATH, VALIDATION_DATASET_PATH, TESTING_DATASET_PATH]
for folder in folders:
    images_path = os.path.join(folder, "images")
    if not os.path.exists(images_path):
        os.makedirs(images_path)
    labels_path = os.path.join(folder, "labels")
    if not os.path.exists(labels_path):
        os.makedirs(labels_path)

print("Folder creation complete!")

In [ ]:
image_files = [
    file for file in os.listdir(TRANSFORMED_DATASET_PATH) if file.endswith(".jpg")
]

num_samples = len(image_files)
num_training = int(TRAINING_RATIO * num_samples)
num_validation = int(VALIDATION_RATIO * num_samples)
num_testing = num_samples - num_training - num_validation

random.shuffle(image_files)

training_files = image_files[:num_training]
validation_files = image_files[num_training : num_training + num_validation]
testing_files = image_files[num_training + num_validation :]

for folder, files in [
    (TRAINING_DATASET_PATH, training_files),
    (VALIDATION_DATASET_PATH, validation_files),
    (TESTING_DATASET_PATH, testing_files),
]:
    for file in files:
        file_label = os.path.splitext(file)[0] + ".txt"

        source_image_path = os.path.join(TRANSFORMED_DATASET_PATH, file)
        destination_image_path = os.path.join(f"{folder}/images", file)
        shutil.move(source_image_path, destination_image_path)

        source_label_path = os.path.join(TRANSFORMED_DATASET_PATH, file_label)
        destination_label_path = os.path.join(f"{folder}/labels", file_label)
        shutil.move(source_label_path, destination_label_path)

print("Splitting complete!")

# 2. Model Loading

In [ ]:
from PIL import Image
from ultralytics import YOLO

MODEL_PATH = "/content/drive/MyDrive/runs4/detect/train9/weights/best.pt"
DATASET_INFORMATION_PATH = "/content/datasets/data.yaml"

NUM_EPOCHS = 7
BATCH_SIZE = 32
IMAGE_SIZE = 640
NUM_WORKERS = 8
DEVICE = 0 # "gpu"


In [ ]:
model = YOLO(MODEL_PATH)

### 2.1. Fine Tunning

In [ ]:
train_params = {
    "data": DATASET_INFORMATION_PATH,
    "imgsz": IMAGE_SIZE,
    "batch": BATCH_SIZE,
    "epochs": NUM_EPOCHS,
    "name": "train",
    "exist_ok": False,
    "weight_decay": 0.0005,
    "optimizer": "SGD",  # Use Adam optimizer
    "device": DEVICE,
    "workers": NUM_WORKERS,
    "resume": False,
}

model.train(**train_params)

### 2.2. Validation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

test_images = "./datasets/test/images"
true_labels = "./datasets/test/labels"

predictions = model.predict(test_images)

for result in predictions:
    # Boxes object for bounding box outputs
    boxes = result.boxes
    # Masks object for segmentation masks outputs
    masks = result.masks
    # Keypoints object for pose outputs
    keypoints = result.keypoints
    # Probs object for classification outputs
    probs = result.probs
    # Oriented boxes object for OBB outputs
    obb = result.obb
    # display to screen
    result.show()
    result.save(filename='result.jpg')

In [ ]:
results = model.val(data="/content/datasets/data.yaml", split='test')  # specify 'test' to evaluate on the test set

# Print results
print(results)




In [ ]:
import os
import numpy as np
import ultralytics
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import cv2


def load_bounding_boxes(txt_path, img_width, img_height):
    boxes = []
    classes = []
    with open(txt_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x_center *= img_width
            y_center *= img_height
            width *= img_width
            height *= img_height
            x1 = x_center - width / 2
            y1 = y_center - height / 2
            x2 = x_center + width / 2
            y2 = y_center + height / 2
            boxes.append([x1, y1, x2, y2])
            classes.append(int(class_id))
    return np.array(boxes), np.array(classes)


def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1, yi1 = max(x1, x1g), max(y1, y1g)
    xi2, yi2 = min(x2, x2g), min(y2, y2g)

    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0


print(f"Mean Average Precision (mAP@0.5): {results.box.maps[0]}")
print(f"Mean Average Precision (mAP@0.5:0.95): {results.box.maps[1]}")


true_classes = []
pred_classes = []

test_images_path = '/content/datasets/test/images'
test_labels_path = '/content/datasets/test/labels'
test_images = os.listdir(test_images_path)

for img_file in test_images:
    img_path = os.path.join(test_images_path, img_file)
    label_path = os.path.join(test_labels_path, os.path.splitext(img_file)[0] + '.txt')

    img = cv2.imread(img_path)
    img_height, img_width, _ = img.shape

    ground_truth_boxes, ground_truth_classes = load_bounding_boxes(label_path, img_width, img_height)

    prediction_results = model(img_path)
    predicted_boxes = prediction_results[0].boxes.xyxy.cpu().numpy()
    predicted_classes = prediction_results[0].boxes.cls.cpu().numpy().astype(int)

    for gt_box, gt_class in zip(ground_truth_boxes, ground_truth_classes):
        if len(predicted_boxes) > 0:
            ious = [iou(gt_box, pred_box) for pred_box in predicted_boxes]
            max_iou_idx = np.argmax(ious)
            if ious[max_iou_idx] > 0.5:  # arbitrary true positive threshold
                true_classes.append(gt_class)
                pred_classes.append(predicted_classes[max_iou_idx])
            else:
                true_classes.append(gt_class)
                pred_classes.append(-1)  # False negative
        else:
            true_classes.append(gt_class)
            pred_classes.append(-1)  # False negative

    for pred_box, pred_class in zip(predicted_boxes, predicted_classes):
        if not any([iou(pred_box, gt_box) > 0.5 for gt_box in ground_truth_boxes]):
            true_classes.append(-1)  # False positive
            pred_classes.append(pred_class)




In [ ]:
labels = [-1, 0, 1]
cm = confusion_matrix(true_classes, pred_classes, labels=labels)
print("Confusion Matrix:")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['background', 'empty', 'occupied'])
disp.plot()
plt.show()

report = classification_report(true_classes, pred_classes, target_names=['background', 'empty', 'occupied'])
print("Classification Report:")
print(report)

In [ ]:
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    confusion_matrix,
    accuracy_score,
)

def printClassResults(model_name, truth, preds):
    print("The Model is: %s" % model_name)
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds, average="weighted"))
    print("The Recall is: %7.4f" % recall_score(truth, preds, average="weighted"))
    print("The F1 score is: %7.4f" % f1_score(truth, preds, average="weighted"))
    print(
        "The Matthews correlation coefficient is: %7.4f"
        % matthews_corrcoef(truth, preds)
    )
    print()


printClassResults("Yolov8 model", true_classes, pred_classes)



In [ ]:

test_images_directory = "./datasets/test/images"

image_files = [f for f in os.listdir(test_images_directory) if f.lower().endswith(".jpg")]

random_image_filename = random.choice(image_files)
random_image_path = os.path.join(test_images_directory, random_image_filename)

results = model.predict(random_image_path)

result = results[0]

print(len(result.boxes))

In [ ]:
backup = random_image_path

In [ ]:
for box in result.boxes:
	label = result.names[box.cls[0].item()]
	coords = [round(x) for x in box.xyxy[0].tolist()]
	prob = round(box.conf[0].item(), 4)
	print("Object: {}\nCoordinates: {}\nProbability: {}".format(label, coords, prob))

In [ ]:
# Original image

Image.open(random_image_path)

In [ ]:
# Image with the result of the model

Image.fromarray(result.plot()[:,:,::-1])